In [1]:
import ray
from ray import tune
import pandas as pd
import os
import matplotlib.pyplot as plt

# Initialize Ray if not already done
if not ray.is_initialized():
    ray.init()

# Specify the experiment directory
experiment_dir = "/home/inigo/ray_results/Blackjack_PPO_AEC"

if os.path.exists(experiment_dir):
    # Get all subdirectories (experiment runs) in the specified directory
    experiment_runs = [d for d in os.listdir(experiment_dir) 
                      if os.path.isdir(os.path.join(experiment_dir, d))]
    
    if experiment_runs:
        # Sort by modification time to get the latest
        experiment_runs.sort(key=lambda x: os.path.getmtime(os.path.join(experiment_dir, x)))
        latest_experiment_path = os.path.join(experiment_dir, experiment_runs[-1])
        
        print(f"Latest experiment: {latest_experiment_path}")
        
        # Load experiment results
        analysis = tune.ExperimentAnalysis(latest_experiment_path)
        
        # Get results dataframe
        results_df = analysis.results_df
        print(f"\nExperiment has {len(results_df)} trials")
        
        # Display basic info about the experiment
        print(f"Columns available: {list(results_df.columns)}")
        
        # Show summary statistics
        print("\nSummary of key metrics:")
        numeric_cols = results_df.select_dtypes(include=['float64', 'int64']).columns
        print(results_df[numeric_cols].describe())
        
        # Plot metrics if available
        if len(numeric_cols) > 0:
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            axes = axes.flatten()
            
            for i, col in enumerate(numeric_cols[:4]):  # Plot first 4 numeric columns
                if i < len(axes):
                    axes[i].plot(results_df[col])
                    axes[i].set_title(f'{col} over trials')
                    axes[i].set_xlabel('Trial')
                    axes[i].set_ylabel(col)
            
            plt.tight_layout()
            plt.show()
        
        # Display best trial
        best_trial = analysis.get_best_trial()
        if best_trial:
            print(f"\nBest trial: {best_trial}")
            print(f"Best trial config: {best_trial.config}")
            print(f"Best trial metrics: {best_trial.last_result}")
    else:
        print(f"No experiment runs found in {experiment_dir}")
else:
    print(f"Directory {experiment_dir} does not exist.")

2025-07-23 18:34:27,406	INFO worker.py:1927 -- Started a local Ray instance.


AttributeError: module 'ray.tune' has no attribute 'list_experiments'

(raylet) [2025-07-23 18:35:27,586 E 225130 225130] (raylet) node_manager.cc:3041: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: e8d52dceae11c3465ecdfaf292b285c92ca229f39ba637b4bbd50db9, IP: 192.168.1.151) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.151`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2025-07-23 18:38:27,592 E 225130 225130] (raylet) node_manager.cc:3041: 9 Workers (tasks / actors) kill